In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/멀티캠퍼스 5조/2.데이터/교통량 데이터/2019-2022도로속도 결측값 처리-0512 진짜.csv")
len(df.columns)

51

In [48]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.head()

,일자,자치구명,00:30:00,01:00:00,01:30:00,02:00:00,02:30:00,03:00:00,03:30:00,04:00:00,...,19:30:00,20:00:00,20:30:00,21:00:00,21:30:00,22:00:00,22:30:00,23:00:00,23:30:00,00:00:00
0,2019.01.01,종로구,25.25,25.25,25.90,25.90,29.18,29.18,30.90,30.90,...,25.90,25.90,27.01,27.01,26.90,26.90,28.33,28.33,29.06,29.06
1,2019.01.01,중구,23.20,23.20,24.08,24.08,27.31,27.31,28.35,28.35,...,23.10,23.10,24.27,24.27,24.90,24.90,26.15,26.15,26.79,26.79
2,2019.01.01,용산구,28.96,28.96,28.39,28.39,29.97,29.97,32.54,32.54,...,30.60,30.60,31.23,31.23,31.96,31.96,32.95,32.95,34.52,34.52
3,2019.01.01,성동구,31.71,31.71,31.73,31.73,33.58,33.58,35.29,35.29,...,29.59,29.59,29.70,29.70,30.40,30.40,31.51,31.51,32.79,32.79
4,2019.01.01,광진구,29.66,29.66,30.74,30.74,32.52,32.52,34.05,34.05,...,28.01,28.01,28.37,28.37,29.19,29.19,30.46,30.46,32.07,32.07


In [49]:
#컬럼명 재설정하기 - 시간을 숫자 순서로 바꾸기.
list1=['일자','자치구명']
list2=[i for i in range(48)]
list3=list1+list2
list3
df.columns=list3
df.columns

Index([  '일자', '자치구명',      0,      1,      2,      3,      4,      5,      6,
            7,      8,      9,     10,     11,     12,     13,     14,     15,
           16,     17,     18,     19,     20,     21,     22,     23,     24,
           25,     26,     27,     28,     29,     30,     31,     32,     33,
           34,     35,     36,     37,     38,     39,     40,     41,     42,
           43,     44,     45,     46,     47],
      dtype='object')

In [ ]:
#결측치 채우기-중앙값으로
import numpy as np
df = df.replace(0, np.NaN)
df.describe()
df=df.fillna(df.median())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  after removing the cwd from sys.path.


In [50]:
#일자 행을 날짜 형식으로 변환 및 열 추가.

df['일자'] = pd.to_datetime(df['일자'])
df['Year'] = df['일자'].dt.year 
df['Month'] = df['일자'].dt.month
df['Day'] = df['일자'].dt.day

In [59]:
#연/월/일 리스트
year_list=[str(i) for i in range(2019,2023)]
month_list=[str(i).zfill(2) for i in range(1,13)]
day_list=[str(i).zfill(2) for i in range(1,32)]

['2019', '2020', '2021', '2022']

In [88]:
#날짜별로 데이터프레임 쪼개기.
for y in year_list:
    globals()['df'+y]=df[df['Year']==int(y)]
    ydf=globals()['df'+y]
    for m in month_list:
        globals()['df'+y+m]=ydf[ydf['Month']==int(m)]
        mdf=globals()['df'+y+m]
        for d in day_list:
            globals()['df'+y+m+d]=mdf[mdf['Day']==int(d)]

In [ ]:
import pandas as pd
total_df=pd.read_csv('/content/drive/MyDrive/멀티캠퍼스 5조/2.데이터/통합데이터_운행차량수추가.csv')

In [ ]:
total_df.insert(12,'차량속도','')

In [ ]:
#각 날짜의 데이터프레임에 가서 해당되는 시간, 자치구명의 차량속도 가져오기.
from tqdm import tqdm
for i in tqdm(range(len(total_df))):
    if total_df.iloc[i,12] !='':
        pass
    else:
        dayprint=str(total_df.iloc[i][6][:4])+str(total_df.iloc[i][6][5:7])+str(total_df.iloc[i][6][8:10])
        value=globals()['df'+dayprint]['자치구명']==total_df.iloc[i][1]
        value2=globals()['df'+dayprint][value]
        a=total_df.iloc[i][8]
        try:
            total_df.at[i,'차량속도']= float(value2[a])
        except:
            total_df.at[i,'차량속도']='시외'


100%|██████████| 3550105/3550105 [33:54<00:00, 1744.81it/s]


In [ ]:
total_df.to_csv('/content/drive/MyDrive/멀티캠퍼스 5조/2.데이터/교통량 데이터/통합데이터 운행차량수 추가에 도로속도 붙임.csv')